In [1]:
import pandas as pd

df1 = pd.read_csv("Cleaned_Link_Review.csv")
df2 = pd.read_csv("Selected_Sentiment_Word.csv")
selected_word = df2["Word"]

listed_selected_word = selected_word.values.tolist()
listed_selected_word

['like',
 'good',
 'best',
 'horror',
 'fun',
 'great',
 'doesnt',
 'little',
 'love',
 'new',
 'entertaining',
 'better',
 'funny',
 'lot',
 'quite',
 'right',
 'bad',
 'bit',
 'dont',
 'real',
 'far',
 'long',
 'kind',
 'want',
 'worth',
 'emotional',
 'interesting',
 'strong',
 'classic',
 'compelling',
 'big',
 'ultimately',
 'original',
 'humor',
 'true',
 'hard',
 'lack',
 'dark',
 'truly',
 'pretty',
 'beautiful',
 'young',
 'cinematic',
 'powerful',
 'feeling',
 'power',
 'perfect',
 'simply',
 'solid',
 'actually',
 'fascinating',
 'violence',
 'trying',
 'entertainment',
 'old',
 'especially',
 'short',
 'effective',
 'sure',
 'enjoyable',
 'easy',
 'final',
 'different',
 'enjoy',
 'fine']

In [15]:
import enchant

english_dict = enchant.Dict("en_US")
type(english_dict)

enchant.Dict

In [2]:
separated_selected_word = '|'.join(listed_selected_word)

df1['separate_match'] = df1["review_content"].str.contains(separated_selected_word, case=False)
df1['selected_word appearance'] = df1['review_content'].str.extract( '({})'.format(separated_selected_word) )

df1
#df1.to_csv("review_content with matching words.csv")


,Unnamed: 0,review_type,review_score,review_content,separate_match,selected_word appearance
0,0,Fresh,3/4,middle school teacher gifted literature value ...,False,NaN
1,1,Fresh,NaN,lightning thief admirable kid fantasy,False,NaN
2,2,Rotten,2.5/5,ok,False,NaN
3,3,Rotten,2/4,there resembling spark film,False,NaN
4,4,Fresh,NaN,truly film age darkly comic stuff mistress mis...,True,truly
...,...,...,...,...,...,...
45741,45741,Fresh,5/5,zootopia rare animated disney film feel like a...,True,like
45742,45742,Fresh,NaN,director michael cacoyannis skill craftsman co...,False,NaN
45743,45743,Fresh,NaN,tension mount break mount unbearable borne rig...,True,right
45744,45744,Fresh,NaN,rousing reconstruction battle rorkes drift,False,NaN


In [3]:
df1.separate_match.value_counts()

True     32473
False    13262
Name: separate_match, dtype: int64

In [4]:
32473/(32473+13262) #Hard Code Percentage from df1.query_match.value_counts()

0.710025144856237

In [5]:
#Replace "fresh" and "rotten" in "review_type" to "1" and "-1"

df1["numeric_review_type"] = df1['review_type'].map({"Fresh": 1 ,'Rotten': -1})

df1["numeric_review_type"] # Checking Purposes

0        1
1        1
2       -1
3       -1
4        1
        ..
45741    1
45742    1
45743    1
45744    1
45745   -1
Name: numeric_review_type, Length: 45746, dtype: int64

In [6]:
#Replace True and False booleans in "separate_match" to "1" and "-1"

x = True
y = False

df1["numeric_separate_match"] = df1['separate_match'].map({ x: int(1) , y: int(-1)})


In [7]:
#Train-Test Preparation

df3 = df1[["numeric_review_type", "numeric_separate_match"]]
df3

,numeric_review_type,numeric_separate_match
0,1,-1.0
1,1,-1.0
2,-1,-1.0
3,-1,-1.0
4,1,1.0
...,...,...
45741,1,1.0
45742,1,-1.0
45743,1,1.0
45744,1,-1.0


In [8]:
#Test/Train Size Estimation 
from sklearn.model_selection import train_test_split
X_data = df3[["numeric_separate_match"]].values
y_data = df3["numeric_review_type"].values

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, 
                                                    test_size = 0.2, 
                                                    random_state=1, 
                                                    stratify = y_data)
X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(X_train, y_train, 
                                                              test_size = 0.2, 
                                                              random_state=1, 
                                                              stratify = y_train)

print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_test.shape:', X_test.shape)
print('y_test.shape:', y_test.shape)
print('Train/Valid/Test sizes:', y_train.shape[0], y_valid.shape[0], y_test.shape[0])

X_train.shape: (36596, 1)
y_train.shape: (36596,)
X_test.shape: (9150, 1)
y_test.shape: (9150,)
Train/Valid/Test sizes: 36596 7320 9150


In [9]:
X_train = X_train.astype('int')
y_train = y_train.astype('int')
X_train_sub = X_train_sub.astype('int')
y_train_sub = y_train_sub.astype('int')
X_valid = X_valid.astype('int')
y_valid = y_valid.astype('int')
X_test = X_test.astype('int')
y_test = y_test.astype('int')

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

C:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [11]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)
print(f"Train Accuracy: {knn.score(X_train_sub, y_train_sub)*100:0.3f}%")
print(f"Valid Accuracy: {knn.score(X_valid, y_valid)*100:0.3f}%")

Train Accuracy: 40.279%
Valid Accuracy: 40.765%


In [12]:
random_forest_model = RandomForestClassifier(criterion='entropy', n_estimators=100,
                                random_state=1)
random_forest_model.fit(X_train, y_train)
print(f"Train Accuracy: {random_forest_model.score(X_train, y_train)*100:0.3f}%")
print(f"Test Accuracy: {random_forest_model.score(X_test, y_test)*100:0.3f}%")

Train Accuracy: 71.371%
Test Accuracy: 71.366%
